In [ ]:
import tensorflow as tf
from   tensorflow import keras
from   tensorflow.keras import regularizers
from   tensorflow.keras import Sequential
from   tensorflow.keras.layers import Dropout, Dense

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [ ]:
import config
import datasets
import models
import trade_processing as tp
import utils

In [ ]:
TICKER   = 'SPY'
WIN_PROB = 10
MARGIN   = 5
PROFIT   = 1 + utils.calculate_fee()

In [ ]:
options_model = models.load_best_model(TICKER, MARGIN, PROFIT)

In [ ]:
# Load the trades
trades_df = datasets.load_dataset(
    ticker=TICKER,
    max_margin=MARGIN,
    min_profit=PROFIT,
    total_datapoints=50000,
    loss_ratio=2,
    loss_pool_multiplier=2,
    verbose=True
).data

In [ ]:
options_model.insert_predictions(trades_df)
trades_df.sort_values(by=['confidence'], ascending=False, inplace=True)

In [ ]:
good_bets = trades_df[trades_df.confidence > WIN_PROB]
winners   = good_bets[(good_bets.max_profit > 1.2)]
print('{:>8} good bets {:>5.1%}, of those {:.2%} winners.'.format(
    good_bets.shape[0], good_bets.shape[0] / trades_df.shape[0], winners.shape[0] / good_bets.shape[0]))

In [ ]:
winners.shape

In [ ]:
good_bets.shape

In [ ]:
winners.minutes_to_expiry.median() / (60 * 24)

In [ ]:
good_bets.max_profit.min()

In [ ]:
good_bets.max_profit.median()

In [ ]:
good_bets.max_profit.max()

In [ ]:
good_bets.hist(
    column=['open_margin', 'max_profit'],
    sharey=True,
    bins=40,
)
plt.show()

In [ ]:
for d in good_bets.description.unique():
    d_bets = good_bets[good_bets.description == d]
    d_total = d_bets.shape[0]
    print('{:>25}: {:>6} ({:>5.1%}), median = {:.3}'.format(
        d, d_total, d_total / good_bets.shape[0], d_bets.max_profit.median()))

In [ ]:
for d in trades_df.description.unique():
    d_bets = trades_df[trades_df.description == d]
    d_total = d_bets.shape[0]
    print('{:>25}: {} ({:.1%})'.format(d, d_total, d_total / trades_df.shape[0]))